In [1]:
import numpy as np
from queue import PriorityQueue as queue
from collections import defaultdict
from faker import Faker
from tabulate import tabulate

fake = Faker()

## Draw Simulation Methods

In [2]:
def init_players(num_players):
    """
    Create a collection of players in the form [player_id, player_name, player_ranking]
    Arguments:
        num_players: Number of players to add to the draw
    Returns:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
    """
    players = []
    id_to_name, id_to_rank = {}, {}
    for n in range(num_players):
        p_id, p_name, p_rank = np.random.randint(1000000, 9999999), fake.name(), np.random.randint(1600, 2000)
        players.append([p_id, p_name, p_rank])
        id_to_name[p_id] = p_name
        id_to_rank[p_id] = p_rank
    return players, id_to_name, id_to_rank

In [3]:
def init_pairings(players):
    """
    Randomly create results for the first round
    Arguments:
        players: (n x 3) numpy array containing [player_id, player_name, player_ranking]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    for n in range(0, len(players), 2):
        a, b = players[n][0], players[n+1][0]
        result = [a, b, np.random.choice([a, b, None], 1, p=(0.45, 0.45, 0.1))[0]]
        results.append(result)
    return results

In [4]:
def run_round(pairings, p_draw=0):
    """
    Randomly create results for a round
    Arguments:
        pairings: List of pairings for the round to be simulated
            (n x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        results: Randomly generated results table
            (n x 3) list containing [player_a_id, player_b_id, match_winner_id]
    """
    results = []
    p_win = (1 - p_draw) / 2
    for pairing in pairings:
        a, b = pairing[0][1], pairing[1][1]
        result = [a, b, np.random.choice([a, b, None], 1, p=(p_win, p_win, p_draw))[0]]
        results.append(result)
        
    return results

## Draw Calculation Methods

In [5]:
def create_draw(players, pairs):
    """
    Create draw for the round
    Agruments:
        players: Array of player details
            (n x 3) list containing [player_id, player_name, player_ranking]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        splits: List of index points where each points bracket finishes
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairings: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(players)

    rankings = defaultdict(int)
    if len(pairs) > 0:
        for pair in pairs:
            rankings[pair[0]] += (pair[0] == pair[2])
            rankings[pair[1]] += (pair[1] == pair[2])
    else:
        for player in players:
            rankings[player[0]] = 0

    standings = np.transpose([list(rankings.keys()), list(rankings.values())])
    standings = np.array(sorted(standings, key=lambda x: x[1], reverse=True))
    
    scores, splits  = get_scores(standings, pairs)
    alloc, pairings = get_allocations(standings, scores)
    
    return standings, splits, scores, alloc, pairings

In [6]:
def get_scores(standings, pairs):
    """
    Assign scores to each possible pairing
    A higher score means that the pairing is more strongly preferred
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        pairs: List of pairings for all rounds to date
            (m x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    Returns:
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
        splits: List of index points where each points bracket finishes
    """
    n_players = len(standings)
    ix = {player_id: pos for pos, player_id in enumerate(standings[:,0])}
    splits = [n for n in range(n_players) if standings[n,1] != standings[n-1,1]]
    
    # Calculate scores for pairings
    scores = np.full((n_players, n_players), -100)
    
    split_1, split_2 = 0, 0
    for split in splits + [n_players]:
        # Assign points for players in same bracket
        scores[split_1:split, split_1:split] = 4
        # Assign points for pull-up bracket
        if split_1 %2 == 1:
            scores[split_2:split_1, split_1:split] = 1
            scores[split_1:split, split_2:split_1] = 1
        split_1, split_2 = split, split_1
    
    # Reset self vs self to -inf
    np.fill_diagonal(scores, -100)

    # Subtract points for prior matchup
    for pair in pairs:
        a, b = ix[pair[0]], ix[pair[1]]
        scores[a,b] -= 1
        scores[b,a] -= 1
        
    return scores, splits

In [44]:
def get_allocations(standings, scores):
    """
    Allocate player pairings
    Arguments:
        standings: List of standings for all players still in the tournament
            (n x 2) list containing [player_id, player_wins]
        scores: Matrix containing the scores for all possible pairings
            (n x n) matrix of floats
    Returns:
        alloc: The matrix form pairing allocations
            (n x n) matrix of integers
        pairs: The list form of pairing allocations
            (n / 2 x 2 x 2) list containing [(player_a_standing, player_a_id), (player_b_standing, player_b_id)]
    """
    n_players = len(standings)
    ix = {pos: player_id for pos, player_id in enumerate(standings[:,0])}
    alloc = np.zeros((n_players, n_players), dtype=int)

    for _ in range(int(n_players / 2)):
        mask = (1 - alloc.sum(axis=1))
        print('M', mask.astype(int))
        #p = 1 - alloc.sum(axis=1)
        #a = np.random.choice(range(n_players),p=p/p.sum())
        # Choose Player A
        p = scores.max() - scores.mean(axis=1)
        p = p * mask
        print('P', p.astype(int))
        a = p.argmax()
        # Choose Player B
        p = np.exp(scores[a])
        p = np.maximum(p * mask, 0)
        b = np.random.choice(range(n_players),p=p/p.sum())
        
        alloc[a, b] = 1
        alloc[b, a] = 1
    
    alloc_ = alloc * np.tri(*alloc.shape)

    pairs = []
    for n in range(n_players):
        if alloc_[n].max() == 1:
            a, b = n, alloc_[n].argmax()
            pairs.append([(a, ix[a]), (b, ix[b])])

    return alloc, pairs

## Draw Printing Methods

In [8]:
def print_standings(standings, id_to_name):
    """
    Print standings for the round in table form
    """
    table = []
    for rank, player in enumerate(standings):
        table.append([int(player[0]), id_to_name[player[0]], player[1], rank+1])
    print('\n', tabulate(table, headers=['ID', 'Name', 'Wins', 'Rank']), '\n')
    

def print_pairings(pairs, id_to_name):
    """
    Print pairings for the round in table form
    """
    table = []
    for table_num, pair in enumerate(pairs):
        b, a = pair[0], pair[1]
        row = [table_num+1, id_to_name[a[1]], int(a[1]), int(a[0])+1, id_to_name[b[1]], int(b[1]), int(b[0])+1]
        table.append(row)
    headers = ['Table #', 'Player 1 - Name', 'ID', 'Rank', 'Player 2 - Name', 'ID', 'Rank']
    print('\n', tabulate(table, headers=headers), '\n')

## Run a sample Tournament
Create a list of players then run through a multi-round tournament and print the results
Each round prints:
- Goodness of fit score: Score for the pairings allocated in the round. The maximum possible score is 4. If a number < 0 comes out, something has probably gone wrong
- Standings: A table of standings of the players (based only on # of wins, no accounting for draws, op wins etc)
- Pairings: A table of pairings for the following round (including player name and standing so you can check if players are being paired with others nearby)

To show all the standings and pairings at once, make sure to select the cell, click on the "Cell" menu, select -> "Current Outputs" and then select "Toggle Scrolling"

The entire history of "standings", "splits", "scores", "allocations", and "pairs" is recorded in the lists *standings_h, splits_h, scores_h, alloc_h, pairs_h*
    
Note: Early stage demo only. This does not support:
- Odd numbers of players and byes
- Allocating pods
- Iterative sampling methods to improve draw quality
- etc

In [47]:
NUMBER_OF_PLAYERS = 32
NUMBER_OF_ROUNDS  = 6
PROBABILITY_OF_DRAW = 0.0001

In [48]:
# Create players
players, id_to_name, id_to_rank = init_players(NUMBER_OF_PLAYERS)
pairings = init_pairings(players)

players

[[4425338, 'Rodney Harris', 1814],
 [1200005, 'Dana Evans', 1792],
 [7713303, 'Nicholas Castro', 1755],
 [3590417, 'Kevin Martin', 1885],
 [2869143, 'Elizabeth Herrera', 1738],
 [8540181, 'Cassidy Harmon', 1758],
 [6882203, 'Juan Henderson', 1881],
 [3154037, 'Jennifer Moran', 1653],
 [3172718, 'Aaron Fletcher', 1985],
 [1274295, 'Jonathan Burnett', 1700],
 [3933243, 'Sarah Hernandez', 1615],
 [9714226, 'Ashley Walker', 1856],
 [4391521, 'Joseph Reeves', 1996],
 [5865190, 'Emily Dixon', 1667],
 [5959588, 'Andrew Cruz', 1799],
 [6937804, 'Taylor Harrison', 1677],
 [2069960, 'Sara Porter', 1907],
 [9421477, 'Troy Adkins', 1825],
 [6947001, 'Roger Cortez', 1600],
 [5095013, 'Jeffery Roy', 1672],
 [5363075, 'Dennis Cruz', 1922],
 [8639369, 'Rachel Hartman', 1711],
 [1807152, 'James Shaw', 1733],
 [5250794, 'Louis Chavez', 1768],
 [6339487, 'Marilyn Castro', 1632],
 [2130138, 'Elizabeth Parsons', 1677],
 [3434278, 'Jennifer Bean', 1992],
 [8669680, 'Bryan Carter', 1604],
 [3343428, 'Rhonda 

In [49]:
results = []
standings_h, splits_h, scores_h, alloc_h, pairs_h = [], [], [], [], []

for rnd in range(NUMBER_OF_ROUNDS):
#for rnd in range(1):
    standings, splits, scores, alloc, pairs = create_draw(players, results)
    for collection, value in zip([standings_h, splits_h, scores_h, alloc_h, pairs_h],
                                 [standings,   splits,   scores,   alloc,   pairs]):
        collection.append(value)
    print('----- Round', rnd + 1, '-----', )
    print('Goodness of Fit:', (scores * alloc).sum() / NUMBER_OF_PLAYERS)
    print_standings(standings, id_to_name)
    print_pairings(pairs, id_to_name)
    result = run_round(pairs, p_draw=PROBABILITY_OF_DRAW)
    results += result

M [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
P [3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
M [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
P [0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3]
M [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
P [0 0 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3]
M [0 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
P [0 0 0 3 3 3 3 3 0 3 3 3 3 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3]
M [0 0 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
P [0 0 0 0 3 3 0 3 0 3 3 3 3 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3]
M [0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
P [0 0 0 0 0 3 0 3 0 3 3 3 0 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3]
M [0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1]
P [0 0 0 0 0 0 0 3 0 3 0 3 0 3 3 3 3 3 3 3 3 0 3 0 3 3 3 3 3 3 3 3]
M [0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1

M [0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
P [ 0 88 88 88  0 88 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74 88 88 88 88 88 88]
M [0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0]
P [ 0 88 88 88  0 88 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74 88  0 88 88 88  0]
M [0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0]
P [ 0  0 88 88  0  0 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74 88  0 88 88 88  0]
M [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0]
P [ 0  0  0  0  0  0 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74 88  0 88 88 88  0]
M [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 0]
P [ 0  0  0  0  0  0 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74  0  0 88 88  0  0]
M [0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0]
P [ 0  0  0  0  0  0 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74 74
 74 74  0  0

In [29]:
standings_h[0], splits_h[0], scores_h[0], alloc_h[0], pairs_h[0]

(array([[5026721,       0],
        [8338010,       0],
        [2706777,       0],
        [4730656,       0],
        [4993862,       0],
        [7138080,       0],
        [5449108,       0],
        [4955421,       0],
        [9619690,       0],
        [1187212,       0],
        [3177525,       0],
        [8792161,       0],
        [2660257,       0],
        [8427855,       0],
        [1714626,       0],
        [7706005,       0],
        [6308784,       0],
        [9953537,       0],
        [6057168,       0],
        [9708248,       0],
        [2747020,       0],
        [9986270,       0],
        [8795647,       0],
        [7967684,       0],
        [6384161,       0],
        [7226961,       0],
        [4403018,       0],
        [8104550,       0],
        [3930921,       0],
        [5415720,       0],
        [7331953,       0],
        [2643320,       0]]),
 [],
 array([[-100,    4,    4, ...,    4,    4,    4],
        [   4, -100,    4, ...,    4,    4,   